### Dataset para el sistema de Recomendacion

In [10]:
# Se cargan las librerias necesarias
import pandas as pd 
import numpy as np 

In [11]:
# Se cargan los datos
reviews = pd.read_csv('reviews_sa.csv', encoding='utf-8')
items = pd.read_parquet('item_clean.parquet', engine='auto')
items['item_id'] = items['item_id'].astype(int)
games = pd.read_csv('games_clean.csv', encoding='utf-8')

Para este dataset se necesita un Rating o valoracion de cada item, para nuestro caso en el dataframe reviews se tienen 2, el sentiment_analysis y rev_recommend. <br>
Se diseña un sistema de valoracion teniendo en cuenta lo anterior de la siguiente manera: <br> 
Cuando: <br>
sentiment_analysis = 0  y rev_recommend = 0  entonces Rating = 1  <br>
sentiment_analysis = 0  y rev_recommend = 1  entonces Rating = 1  <br>
sentiment_analysis = 1  y rev_recommend = 0  entonces Rating = 2  <br>
sentiment_analysis = 1  y rev_recommend = 1  entonces Rating = 3  <br>
sentiment_analysis = 2  y rev_recommend = 0  entonces Rating = 4  <br>
sentiment_analysis = 2  y rev_recommend = 1  entonces Rating = 5  <br>


Volviendo lo anterior codigo y aplicandolo a games tenemos:

In [12]:
# Se definen las condiciones y los valores a asignar
conditions = [
    (reviews['sentiment_analysis'] == 0) & (reviews['rev_recommend'] == 0),
    (reviews['sentiment_analysis'] == 0) & (reviews['rev_recommend'] == 1),
    (reviews['sentiment_analysis'] == 1) & (reviews['rev_recommend'] == 0),
    (reviews['sentiment_analysis'] == 1) & (reviews['rev_recommend'] == 1),
    (reviews['sentiment_analysis'] == 2) & (reviews['rev_recommend'] == 0),
    (reviews['sentiment_analysis'] == 2) & (reviews['rev_recommend'] == 1)
]

# Valores para cada condición
values = [1, 1, 2, 3, 4, 5]  

# Se aplican las condiciones y se asignan los valores usando np.select()
reviews['Rating'] = np.select(conditions, values, default=None)
reviews.head()

,user_id,user_url,variable,rev_item_id,rev_helpful,rev_recommend,año,sentiment_analysis,Rating
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,1250,No ratings yet,True,2011,2,5
1,js41637,http://steamcommunity.com/id/js41637,0,251610,15 of 20 people (75%) found this review helpful,True,2014,2,5
2,evcentric,http://steamcommunity.com/id/evcentric,0,248820,No ratings yet,True,invalido,2,5
3,doctr,http://steamcommunity.com/id/doctr,0,250320,2 of 2 people (100%) found this review helpful,True,2013,2,5
4,maplemage,http://steamcommunity.com/id/maplemage,0,211420,35 of 43 people (81%) found this review helpful,True,2014,1,3


Se hacen  tablas auxiliares  para tomar los datos necesarios para el sistema de recomendacion

In [13]:
reviews_aux =  reviews[['rev_item_id', 'user_id', 'Rating']]
reviews_aux = reviews_aux.rename(columns={'rev_item_id':'item_id'})

In [14]:
# Se usa la items para traer los nombres de los juegos, se usa esta tabla porque tiene mayor numero de juegos 
items_aux = items[['item_id', 'item_name']]
items_aux = items_aux.drop_duplicates(subset=['item_id'])

Se hace la union de las tablas

In [15]:
training = reviews_aux.merge(items_aux, on='item_id')
training = training.drop_duplicates()
training.head()

,item_id,user_id,Rating,item_name
0,1250,76561197970982479,5,Killing Floor
1,1250,EndAtHallow,1,Killing Floor
2,1250,76561198077432581,5,Killing Floor
3,1250,76561198057958244,5,Killing Floor
4,1250,46366536564574576346346546,5,Killing Floor


In [16]:
training.item_id.nunique()

3296

In [17]:
training.to_parquet('training.parquet')
print(f'Se guardó el archivo')

Se guardó el archivo
